# Get Data and Code from Github

In [1]:
import os
directory = "seminar-dlmb-2024-winter-public"

if os.path.exists(directory) and os.listdir(directory):
    print(f"Directory {directory} exists and is non-empty.")
else:
    print(f"Directory {directory} does not exist or is empty.")
    !!git clone https://github.com/hzi-bifo/seminar-dlmb-2024-winter-public.git

Directory seminar-dlmb-2024-winter-public does not exist or is empty.


# Install Some Stuff

In [2]:
%pip install biopython seaborn
%pip install biopython
%pip install seaborn
%pip install -q biopython transformers datasets huggingface_hub accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
%pip install pytorch torchvision cudatoolkit=10.0 -c pytorch

!git clone https://github.com/jerryji1993/DNABERT
!cd DNABERT
%pip install -r --editable .
!cd examples
%pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'
Cloning into 'DNABERT'...
remote: Enumerating objects: 778, done.
remote: Counting objects: 100% (778/778), done.
remote: Compressing objects: 100% (348/348), done.
remote: Total 778 (delta 420), reused 721 (delta 407), pack-reused 0 (from 0)
Receiving objects: 100% (778/778), 11.66 MiB | 18.99 MiB/s, done.
Resolving deltas: 100% (420/420), done.
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
/bin/bash: line 1: cd: examples: No such file or directory
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


# Copy of Custom Classes (see git)

In [4]:
import random
import sys

class SEQData():
    def __init__(self, seed):
        random.seed(seed)
        # make sure that "/seminar-dlmb-2024-winter-public" exists

        sys.path.insert(0, 'seminar-dlmb-2024-winter-public/src/')

        from amr.amr_utility import load_gene_data, create_gene_datasets

        create_gene_datasets("seminar-dlmb-2024-winter-public/", "seminar-dlmb-2024-winter-public/data/ds1")
        ds = load_gene_data("seminar-dlmb-2024-winter-public/data/ds1", "Staphylococcus_aureus_cefoxitin", "pbp4")

        sequences_train = [x[1] for x in ds["train"]]
        labels_train = [x[2] for x in ds["train"]]

        sequences_test = [x[1] for x in ds["test"]]
        labels_test = [x[2] for x in ds["test"]]

        sequences = sequences_train + sequences_test
        labels = labels_train + labels_test

        class_0 = [(seq, label) for seq, label in zip(sequences, labels) if label == 0]
        class_1 = [(seq, label) for seq, label in zip(sequences, labels) if label == 1]

        random.shuffle(class_0)
        random.shuffle(class_1)

        test_data = class_0[:7] + class_1[:7]
        leftover_class_0 = class_0[7:]
        leftover_class_1 = class_1[7:]

        val_data = leftover_class_0[:7] + leftover_class_1[:7]
        leftover_class_0 = leftover_class_0[7:]
        leftover_class_1 = leftover_class_1[7:]

        train_data = leftover_class_0 + leftover_class_1

        self.test_sequences = [seq for seq, label in test_data]
        self.test_labels = [label for seq, label in test_data]

        self.val_sequences = [seq for seq, label in val_data]
        self.val_labels = [label for seq, label in val_data]

        self.train_sequences = [seq for seq, label in train_data]
        self.train_labels = [label for seq, label in train_data]

In [5]:
from itertools import product

class SEQTokenizer_Kmers():
    """Custom tokenizer for DNA sequences with k-mers."""
    def __init__(self, k):
        self.k = k

        self.vocab = [''.join(kmer) for kmer in product('ATCG', repeat=k)]
        self.token_to_id = {token: i for i, token in enumerate(self.vocab)}

    def encode(self, sequence):
        kmers = [sequence[i:i+self.k] for i in range(len(sequence) - self.k + 1)]
        return [self.token_to_id[kmer] for kmer in kmers if kmer in self.token_to_id]

    def token_id_tensor_to_id_array(self, token_id_tensor):
        return [token_id.item() for token_id in token_id_tensor]


In [6]:
import torch

class SEQDataset(torch.utils.data.Dataset):
    def __init__(self, sequences, labels, tokenizer):
        self.sequences = sequences
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        label = self.labels[idx]
        tokens = self.tokenizer.encode(sequence)
        return torch.tensor(tokens, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# Load, Tokenize and save Data as Dataset

In [7]:
import random

random.seed(42)

In [8]:
tokenizer = SEQTokenizer_Kmers(6)

#loat genetic sequence data from the seminar-dlmb-2024-winter-public repository
data = SEQData(42)

# create datasets of type torch.utils.data.Dataset
train_dataset = SEQDataset(data.train_sequences, data.train_labels, tokenizer)
val_dataset = SEQDataset(data.val_sequences, data.val_labels, tokenizer)
test_dataset = SEQDataset(data.test_sequences, data.test_labels, tokenizer)


def create_dict(dataset, name):
    dataset_sequences = []
    dataset_labels = []

    print(f"Creating dict for set '{name}'...")

    for token, label in dataset:
        tokens = tokenizer.token_id_tensor_to_id_array(token)

        tokens = tokens[:512]

        dataset_sequences.append(tokens)
        dataset_labels.append(label.item())

    random.shuffle(dataset_sequences) # with same seed!!!
    random.shuffle(dataset_labels)

    return dataset_sequences, dataset_labels

In [9]:
train_seq, train_y = create_dict(train_dataset, 'train')
val_seq, val_y = create_dict(val_dataset, 'validation')
test_seq, test_y = create_dict(test_dataset, 'test')

Creating dict for set 'train'...
Creating dict for set 'validation'...
Creating dict for set 'test'...


In [10]:
train_dict = {'input_ids': train_seq}
val_dict = {'input_ids': val_seq}
test_dict = {'input_ids': test_seq}

In [11]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)
test_dataset = Dataset.from_dict(test_dict)

train_dataset = train_dataset.add_column('labels', train_y)
val_dataset = val_dataset.add_column('labels', val_y)
test_dataset = test_dataset.add_column('labels', test_y)

print(train_dataset[0])

{'input_ids': [1792, 3072, 1, 5, 21, 84, 336, 1345, 1284, 1041, 70, 281, 1124, 401, 1605, 2324, 1105, 326, 1304, 1121, 390, 1560, 2145, 389, 1557, 2133, 340, 1361, 1351, 1309, 1141, 469, 1876, 3408, 1346, 1288, 1057, 133, 532, 2128, 323, 1293, 1076, 209, 837, 3348, 1105, 327, 1308, 1138, 456, 1826, 3210, 552, 2209, 644, 2577, 2119, 286, 1144, 482, 1928, 3616, 2179, 526, 2105, 231, 926, 3705, 2532, 1936, 3650, 2312, 1059, 141, 567, 2268, 882, 3531, 1837, 3252, 720, 2882, 3338, 1066, 170, 681, 2727, 2717, 2676, 2514, 1864, 3360, 1155, 526, 2104, 227, 910, 3640, 2272, 896, 3585, 2054, 24, 96, 385, 1540, 2065, 71, 287, 1149, 501, 2004, 3921, 3399, 1310, 1144, 483, 1935, 3645, 2293, 981, 3927, 3421, 1398, 1499, 1903, 3518, 1785, 3047, 3998, 3704, 2529, 1924, 3602, 2123, 300, 1200, 706, 2826, 3115, 172, 690, 2763, 2860, 3251, 717, 2871, 3294, 889, 3559, 1949, 3701, 2516, 1872, 3393, 1287, 1053, 117, 468, 1875, 3405, 1334, 1240, 864, 3456, 1538, 2057, 39, 159, 636, 2546, 1992, 3872, 3201, 517

# Compute Class Weights

In [12]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Get labels from the training dataset
labels = train_dataset["labels"]

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",  # Automatically calculate balanced weights
    classes=np.unique(labels),  # Unique classes in the dataset
    y=labels                  # Labels of the dataset
)

# Convert to PyTorch tensor
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)
print("Class Weights:", class_weights_tensor)


Class Weights: tensor([2.3462, 0.6354])


# Define Transfomer Model

In [13]:
from transformers import AutoModelForSequenceClassification

# Define a custom model with weighted loss
class WeightedModel(torch.nn.Module):
    def __init__(self, model_name, class_weights):
        super().__init__()
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(class_weights))
        self.loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        if labels is not None:  # Compute weighted loss if labels are provided
            loss = self.loss_fn(logits, labels)
            return {"loss": loss, "logits": logits}
        else:
            return {"logits": logits}

# Initialize the model with class weights
model = WeightedModel('zhihan1996/DNA_bert_6', class_weights_tensor)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/359M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNA_bert_6 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Fine-Tuning of the Model

In [16]:
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding

# Define the data collator
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNA_bert_6")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    logging_strategy="steps",     # Log training metrics every few steps
    logging_steps=1,             # Log every 10 steps
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=25,
    weight_decay=0.01,
    save_total_limit=2,
    report_to="none",             # Disable logging to external tools like WandB
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-16-094f3c3a7fdd>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.718700,0.740743
2,0.648200,0.756176
3,0.704000,0.698363
4,0.671200,0.697228
5,0.677900,0.694992
6,0.717400,0.701412
7,0.623000,0.715462
8,0.633100,0.724039
9,0.654700,0.754999
10,0.704200,0.752160


TrainOutput(global_step=100, training_loss=0.675241961479187, metrics={'train_runtime': 60.623, 'train_samples_per_second': 50.311, 'train_steps_per_second': 1.65, 'total_flos': 0.0, 'train_loss': 0.675241961479187, 'epoch': 25.0})

# Evaluation of the Model

In [18]:
# Evaluate the model
results = trainer.evaluate(eval_dataset=val_dataset)
print(results)

{'eval_loss': 0.720199465751648, 'eval_runtime': 0.0907, 'eval_samples_per_second': 154.334, 'eval_steps_per_second': 11.024, 'epoch': 25.0}


In [19]:
predictions = trainer.predict(test_dataset=test_dataset)
print(predictions.predictions)  # Raw logits


import numpy as np

# Get predicted class labels from logits
predicted_classes = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

# Display the predicted class labels
print("Predicted Classes:", predicted_classes)
print("True Classes:", predictions.label_ids)    # True labels

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def compute_metrics(true_labels, predicted_classes):
    precision = precision_score(true_labels, predicted_classes, average="binary")
    recall = recall_score(true_labels, predicted_classes, average="binary")
    f1 = f1_score(true_labels, predicted_classes, average="binary")
    accuracy = accuracy_score(true_labels, predicted_classes)

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

# Call the function
metrics = compute_metrics(true_labels, predicted_classes)

[[-0.19189407  0.14240378]
 [ 0.1358451   0.01074923]
 [ 0.14235814  0.02645456]
 [-0.20289828  0.15455388]
 [ 0.14235814  0.02645456]
 [ 0.14729264  0.02621548]
 [-0.16945182  0.1936451 ]
 [ 0.14235814  0.02645456]
 [-0.2004354   0.15956615]
 [ 0.13493775  0.02910375]
 [-0.20170954  0.14722264]
 [-0.20456882  0.15345126]
 [-0.17107749  0.19276169]
 [ 0.15474094  0.03697062]]
Predicted Classes: [1 0 0 1 0 0 1 0 1 0 1 1 1 0]
True Classes: [0 1 0 1 1 0 0 0 1 1 0 0 1 1]
Precision: 0.4286
Recall: 0.4286
F1 Score: 0.4286
Accuracy: 0.4286
